[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/browndw/humanities_analytics/blob/main/mini_labs/Mini_Lab_06_Collocations.ipynb)

# Mini Lab 6: Collocations

We're going to use the same libraries and introductory steps in our processing pipeline.

For details about the package and it's functions see: <https://docuscospacy.readthedocs.io/en/latest/docuscope.html>

If you'd like to explore what this library does in an interactive online interface, you can go to: <https://docuscope-ca.eberly.cmu.edu/>

## Prepare your data

In this lab, we'll also be reviewing the steps for loading text data directly from your Google Drive. To prepare, follow these steps:


1.   Download the `ya_corpus` from Canvas.
2.   This is a zipped file, so you will need to unzip it. On a Mac, just double-click the file. On Windows, right-click on the zip file you want to expand. Then, from the context menu, choose "Extract All".
3.   Make a `data` folder on your Google Drive in a location that is sensible to you.
4.  In that new directory, from Drive click on `+ New` then `Folder upload`. Find the unzipped `ya_corpus` and upload it.

---
If you're using the lab and don't have access to Canvas, you can read in some preprocessed data, then filter it in place of code chunk 11.

```{python}
twilight_tokens = pl.read_parquet('https://github.com/browndw/humanities_analytics/raw/refs/heads/main/data/data_tables/ya_tokens.parquet')

twilight_tokens = twilight_tokens.filter(pl.col("doc_id").str.contains("Twilight"))
```


## Install the libraries

Note that the capture decorator simply supresses the installation output.

In [1]:
%%capture
!pip install docuscospacy>=0.3

## Install the model

Next we'll install the model.

In [2]:
%%capture
!pip install "en_docusco_spacy @ https://huggingface.co/browndw/en_docusco_spacy/resolve/main/en_docusco_spacy-any-py3-none-any.whl"

## Mount Drive

Connect to your Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the libraries

We'll need these for our proceessing pipeline (docuscospacy, spacy) wrangle data frames (polars), generate and maipulate tables (great_tables) and create plots (matplotlib).

In [3]:
import docuscospacy as ds
import polars as pl
import spacy

## Import data

Once you've added the ya_corpus folder to your Drive, you can read in the **Twilight** texts.

Change the path (to JUST the Twilight directory), remove the comment (#) and run...

In [ ]:
# twilight_corpus = ds.corpus_from_folder("/content/drive/MyDrive/76-380-780 MiHA/Mini Labs/data/ya_corpus/Twilight")

The resulting table should have a `doc_id` column and a `text` column. This is conventional formatting for processing textual data.

In [ ]:
twilight_corpus.head()

doc_id,text
str,str
"""Meyer_BreakingDawn.txt""","""PREFACE I'd had more than my …"
"""Meyer_Eclipse.txt""","""PREFACE ALL OUR ATTEMPTS AT SU…"
"""Meyer_NewMoon.txt""","""PREFACE I FELT LIKE I WAS TRA…"
"""Meyer_Twilight.txt""","""Preface I'd never given much …"


For the purposes of this lab, let's subset the corpus so our processing will be quicker. We'll filter the corpus for the first novel, *Twilight*.

In [ ]:
twilight_subcorpus = twilight_corpus.filter(pl.col("doc_id").str.contains("Twilight"))

## Load the model

As with any processing task of this kind, we will first need to load a model instance.

In [ ]:
nlp = spacy.load("en_docusco_spacy")

## Process the corpus

Now, we pass our data frame `df` to the nlp model that we just loaded using the `docuscope_parse` function. This is the most computationally intensive part of the process (and it takes the longest). For this sample corpus, processing should take a couple of minutes.

If you're working with a medium to large corpus, you'd typically want to down-sample the full corpus and check to make everything is working before processing the full data set.

Down-sample our `df` would be easy using the `sample` function in [`polars`](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.sample.html).

In [ ]:
twilight_tokens = ds.docuscope_parse(twilight_subcorpus, nlp_model=nlp, n_process=4)

### Peek at the tokens

Now we can check what we've generated.

In [ ]:
twilight_tokens.head()

doc_id,token,pos_tag,ds_tag,pos_id,ds_id
str,str,str,str,u32,u32
"""Meyer_Twilight.txt""","""Preface ""","""NN1""","""InformationPlace""",1,1
"""Meyer_Twilight.txt""","""I""","""PPIS1""","""FirstPerson""",2,2
"""Meyer_Twilight.txt""","""'d ""","""VM""","""FirstPerson""",3,2
"""Meyer_Twilight.txt""","""never ""","""RR""","""ForceStressed""",4,3
"""Meyer_Twilight.txt""","""given ""","""VVN""","""ForceStressed""",5,3


## Collocations

The `coll_table` function produces collocations for a specified token in a **docuscope tokens** object. In addition to a token, a span or window can be specified (as given by a number of words **preceding** and **following** the **node word**). The default is 4 to the left and 4 to the right.

The formula for calculating MI is based on the observed vs. expected frequency within a span or window:

$$log_{2} \frac{O_{11}}{E_{11}}$$

However, this measure (pointwise mutual information) is sensitive to rare or infrequent words. Alternate versions of this measure (PMI2, PMI3 and normalized PMI) tend to perform better. The default is normalized PMI (or npmi). But any of these statistics can be specified.

## Generating a table of collocates

Let's start be generating a table of collocates around the node word *me*, keeping in mind that the novel is written in the first person.

In [5]:
me_collocates = ds.coll_table(twilight_tokens, node_word='me')

And check the results.

In [6]:
me_collocates.head()

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""trust""","""VV0""",10,8,0.487659
"""he""","""PPHS1""",482,2784,0.471936
"""tell""","""VV0""",23,34,0.463423
"""astonished""","""VVN""",2,1,0.459082
"""coherent""","""JJ""",2,1,0.459082


Like a keyness table. A collocations table can be difficult to make sense out of. Typically, results require sorting and filtering to locate interesting or explanatory patterns.

In [ ]:
me_collocates.filter(pl.col("Tag").str.starts_with("NN")).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""oath""","""NN1""",2,1,0.459082
"""jump""","""NN1""",5,4,0.454196
"""think""","""NN1""",3,2,0.449514
"""english""","""NN1""",7,9,0.421259
"""route""","""NN1""",2,2,0.39617
"""arms""","""NN2""",21,55,0.392562
"""absurdities""","""NN2""",1,1,0.372722
"""addiction""","""NN1""",1,1,0.372722
"""ambivalent""","""NN1""",1,1,0.372722


In [7]:
me_collocates.filter(pl.col("Tag").str.starts_with("NN"),
                     pl.col("Freq Total") > 10).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""arms""","""NN2""",21,55,0.392562
"""door""","""NN1""",34,166,0.339577
"""mom""","""NN1""",16,62,0.336793
"""chest""","""NN1""",8,25,0.334889
"""surprise""","""NN1""",7,23,0.325174
"""step""","""NN1""",6,19,0.323841
"""eyes""","""NN2""",70,497,0.322257
"""look""","""NN1""",11,44,0.319834
"""minute""","""NNT1""",8,36,0.29703


In [ ]:
me_collocates.filter(pl.col("Tag") == "VVG",
                     pl.col("Freq Total") > 5).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""staring""","""VVG""",24,62,0.400315
"""telling""","""VVG""",7,13,0.383602
"""speaking""","""VVG""",7,14,0.376012
"""holding""","""VVG""",10,23,0.375412
"""waiting""","""VVG""",18,64,0.351044
"""pulling""","""VVG""",7,18,0.350276
"""touching""","""VVG""",5,13,0.337515
"""glancing""","""VVG""",4,10,0.334019
"""opening""","""VVG""",3,7,0.331466


It can also be useful to generate a collocations table for a contrasting node word for comparision. We could, for example, make a similar table for collocations around *edward*:

In [ ]:
edward_collocates = ds.coll_table(twilight_tokens, node_word='edward')

And check the result...

In [ ]:
edward_collocates.head()

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""cullen""","""NP1""",31,61,0.599097
"""behave""","""VV0""",2,1,0.57441
"""'s""","""GE""",71,347,0.543609
"""snarled""","""VVD""",2,2,0.511499
"""tracking""","""VVG""",2,2,0.511499


In [ ]:
edward_collocates.filter(pl.col("Tag").str.starts_with("NN")).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""glare""","""NN1""",3,5,0.482906
"""belts""","""NN2""",1,1,0.481224
"""birthday""","""NN1""",1,1,0.481224
"""confessions""","""NN2""",1,1,0.481224
"""cycle""","""NN1""",1,1,0.481224
"""dammit""","""NN1""",1,1,0.481224
"""denial""","""NN1""",1,1,0.481224
"""earnestness""","""NN1""",1,1,0.481224
"""elation""","""NN1""",1,1,0.481224


In [ ]:
edward_collocates.filter(pl.col("Tag").str.starts_with("NN"),
                         pl.col("Freq Total") > 10).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""look""","""NN1""",7,44,0.388868
"""step""","""NN1""",3,19,0.357109
"""shadows""","""NN2""",2,11,0.356772
"""biology""","""NN1""",3,21,0.347678
"""matter""","""NN1""",2,13,0.341609
"""gaze""","""NN1""",3,24,0.335095
"""engine""","""NN1""",3,25,0.331249
"""boyfriend""","""NN1""",2,15,0.328621
"""horror""","""NN1""",2,16,0.322763


In [ ]:
edward_collocates.filter(pl.col("Tag") == "VVG",
                         pl.col("Freq Total") > 5).head(9)

Token,Tag,Freq Span,Freq Total,MI
str,str,u32,u32,f64
"""glancing""","""VVG""",3,10,0.417591
"""standing""","""VVG""",5,25,0.398569
"""grinning""","""VVG""",2,10,0.365422
"""speaking""","""VVG""",2,14,0.334883
"""staring""","""VVG""",6,62,0.33271
"""glaring""","""VVG""",1,6,0.328225
"""imagining""","""VVG""",1,6,0.328225
"""missing""","""VVG""",1,6,0.328225
"""wondering""","""VVG""",3,27,0.323997


## Tables of clusters

You can also generate n-grams anchored by a token in a specified position (also called "clusters". Here, we'll generate 2-grams with *me* in the 2nd position.

In [8]:
me_clusters = ds.clusters_by_token(twilight_tokens, "me", node_position=2, span=2)

And check the result...

In [9]:
me_clusters.head()

Token_1,Token_2,Tag_1,Tag_2,AF,RF,Range
str,str,str,str,u32,f64,f64
"""at""","""me""","""II""","""PPIO1""",145,1189.538623,100.0
"""to""","""me""","""II""","""PPIO1""",107,877.797467,100.0
"""for""","""me""","""IF""","""PPIO1""",84,689.11203,100.0
"""with""","""me""","""IW""","""PPIO1""",62,508.630308,100.0
"""behind""","""me""","""II""","""PPIO1""",39,319.944871,100.0


Here, again, it is usually necessary to perform some kind of filtering or selection....

In [ ]:
me_clusters.filter(pl.col("Tag_1") == "VVG")

Token_1,Token_2,Tag_1,Tag_2,AF,RF,Range
str,str,str,str,u32,f64,f64
"""watching""","""me""","""VVG""","""PPIO1""",9,73.833432,100.0
"""telling""","""me""","""VVG""","""PPIO1""",7,57.426002,100.0
"""pulling""","""me""","""VVG""","""PPIO1""",6,49.222288,100.0
"""holding""","""me""","""VVG""","""PPIO1""",6,49.222288,100.0
"""making""","""me""","""VVG""","""PPIO1""",5,41.018573,100.0
…,…,…,…,…,…,…
"""sling""","""me""","""VVG""","""PPIO1""",1,8.203715,100.0
"""connecting""","""me""","""VVG""","""PPIO1""",1,8.203715,100.0
"""facing""","""me""","""VVG""","""PPIO1""",1,8.203715,100.0


Now, let's generate 3-grams with the patterns: *he * me* and *I * him*.

First, well generate 3-grams with *he* in the 1st position:

In [ ]:
he_me_clusters = ds.clusters_by_token(twilight_tokens, "he", node_position=1, span=3)

Then, we'll filter those with me in the 3rd:

In [ ]:
he_me_clusters = he_me_clusters.filter(pl.col("Token_3") == "me")
he_me_clusters.head(10)

Token_1,Token_2,Token_3,Tag_1,Tag_2,Tag_3,AF,RF,Range
str,str,str,str,str,str,u32,f64,f64
"""he""","""reminded""","""me""","""PPHS1""","""VVD""","""PPIO1""",9,73.833432,100.0
"""he""","""ignored""","""me""","""PPHS1""","""VVD""","""PPIO1""",7,57.426002,100.0
"""he""","""pulled""","""me""","""PPHS1""","""VVD""","""PPIO1""",6,49.222288,100.0
"""he""","""told""","""me""","""PPHS1""","""VVD""","""PPIO1""",6,49.222288,100.0
"""he""","""held""","""me""","""PPHS1""","""VVD""","""PPIO1""",5,41.018573,100.0
"""he""","""walked""","""me""","""PPHS1""","""VVD""","""PPIO1""",4,32.814859,100.0
"""he""","""caught""","""me""","""PPHS1""","""VVD""","""PPIO1""",4,32.814859,100.0
"""he""","""asked""","""me""","""PPHS1""","""VVD""","""PPIO1""",4,32.814859,100.0
"""he""","""threw""","""me""","""PPHS1""","""VVD""","""PPIO1""",3,24.611144,100.0


And now for *I * him*:

In [ ]:
i_him_clusters = ds.clusters_by_token(twilight_tokens, "I", node_position=1, span=3)
i_him_clusters = i_him_clusters.filter(pl.col("Token_3") == "him")
i_him_clusters.head(10)

Token_1,Token_2,Token_3,Tag_1,Tag_2,Tag_3,AF,RF,Range
str,str,str,str,str,str,u32,f64,f64
"""i""","""reminded""","""him""","""PPIS1""","""VVD""","""PPHO1""",12,98.444576,100.0
"""i""","""assured""","""him""","""PPIS1""","""VVD""","""PPHO1""",7,57.426002,100.0
"""i""","""told""","""him""","""PPIS1""","""VVD""","""PPHO1""",6,49.222288,100.0
"""i""","""warned""","""him""","""PPIS1""","""VVD""","""PPHO1""",6,49.222288,100.0
"""i""","""watched""","""him""","""PPIS1""","""VVD""","""PPHO1""",6,49.222288,100.0
"""i""","""saw""","""him""","""PPIS1""","""VVD""","""PPHO1""",5,41.018573,100.0
"""i""","""ignored""","""him""","""PPIS1""","""VVD""","""PPHO1""",4,32.814859,100.0
"""i""","""felt""","""him""","""PPIS1""","""VVD""","""PPHO1""",3,24.611144,100.0
"""i""","""gave""","""him""","""PPIS1""","""VVD""","""PPHO1""",2,16.407429,100.0


Once, you've identified promising data, it is generally easier to work with it outside of the coding environment.

You can write any table by using `write_csv` and specifying a path to a Drive folder:

In [ ]:
# me_collocates.write_csv("/content/drive/MyDrive/76-380-780 MiHA/Mini Labs/output/me_collocates.csv")

## ✅ Task 1

With a team or on your own, take one or more of tables we've generated above (or generate new ones on your own). Write the table (or tables) to your Drive.

Filter, hightlight, or just eyeball the data. Identify a promising like or inquiry, a pattern, or a story. Prepare to share your findings with the class.

In [ ]:
# Your code here

In [ ]:
# Your code here

In [ ]:
# Your code here

## 🤔 Discussion Questions

1. **Interpreting Association Measures**: The MI (Mutual Information) score tells you how strongly two words attract each other. Looking at the "me" collocates, why might words like "trust" and "tell" have high MI scores? What does this reveal about how the narrator relates to others?

2. **Context Window Choices**: We used the default span of 4 words before and after the node word. How might changing this window (e.g., 2 words or 8 words) affect which collocates appear most significant? What linguistic phenomena require wider vs. narrower windows?

3. **Rare vs. Frequent Collocates**: Notice that some high-MI collocates appear infrequently (low "Freq Total"). Why does normalized PMI help with this problem? When might you want to filter by minimum frequency threshold?

4. **Comparing Node Words**: We looked at "me" (narrator) and "edward" (love interest). What do the different collocational patterns reveal about how these characters are portrayed? What verbs, nouns, or adjectives cluster differently around each?

5. **Clusters vs. Collocates**: The `clusters_by_token` function finds fixed n-grams (like "told me" or "asked me"), while `coll_table` finds any words within a window. When would you want one versus the other? What research questions does each method answer?

6. **From Patterns to Interpretation**: You found patterns like "he * me" appearing frequently. What does this grammatical structure suggest about narrative agency or power dynamics in the text? How would you move from computational observation to literary interpretation?

## 💡 Experimentation Ideas

1. **Character Network Analysis**: Generate collocations for multiple character names (bella, edward, jacob, alice, etc.). Which characters collocate with which others? Can you map a social network from collocational patterns?

2. **Verb Pattern Comparison**: Filter collocates by verb tags (VVG, VVN, VV0) for different node words. Do certain characters get associated more with action verbs vs. stative verbs? What does this reveal about agency?

3. **Symmetry Analysis**: Compare "me * edward" clusters with "edward * me" clusters. Are the patterns symmetric? What verbs or prepositions appear between them in each direction? What does asymmetry suggest?

4. **Genre-Specific Collocates**: If you process the full YA corpus (multiple books), compare collocations across different series or genres. Do romance YA novels have different collocational patterns than dystopian YA?

5. **Temporal Clusters**: Generate 3-grams and 4-grams anchored by time-related words (day, night, moment, suddenly). What temporal structures characterize the narrative? Are there repeated temporal formulas?

6. **Negative Space**: Look for collocates that you'd *expect* to find but don't appear. For instance, if "love" doesn't collocate strongly with a character name, what does that absence suggest? Sometimes what's missing is as informative as what's present.